# 計算物理III 数値磁性体物性入門

## 6.2.3 部分ボルツマン因子の計算

### 1次元XXXハイゼンベルグモデルのn次近似分配関数を求める

ハミルトニアン
$$
\mathcal{H}
= \sum_{j=1}^{L} \mathcal{H}_{j,j+1}
= \sum_{j} \left\{
   -2J_{xy}\left(S_j^x S_{j+1}^x + S_j^y S_{j+1}^y \right)
   -2J_{z} S_j^z S_{j+1}^z
  \right\}.
$$

$L=4, n=2, \beta=1, J=1$でチェッカーボード分解してZnを求めるコード。電子配置は$2^{16}=65535$通りあり、それら全てのパターンに対して、部分ボルツマン因子の行列から成分を取り出して各黒マスごとの積を計算し、和を取って計算した。

おそらくもっと効率の良い計算方法（量子転送行列法？）があるはずだが、Theoryに従ってそのまま計算してみた。

結果は、90パターンの電子配置について、積が0以外になり、合計は$Z_n=44.627744732937494$となった。ChatGPTが出した解の一つと一致しているが確信が持てない...

（追記）
黒マスは許容配置、白マスが不許容配置を含んで90パターン。全マスが許容配置になるのは30パターンのみ。どちらで計算すべきかわからない。。 => 黒マスだけ許容配置になっていればOK

In [1]:
from re import A
import numpy as np
import math

J = 1
beta = 1
L = 4
n = 2

a_in_rho = math.exp(beta*J/(2*n))
b_in_rho = math.exp(-beta*J/(2*n))*math.cosh(beta*J/n)
c_in_rho = math.exp(-beta*J/(2*n))*math.sinh(beta*J/n)

def all_4x4_binary_matrices_array():
  """
  0..65535 を2進展開して 4x4 に並べた全パターンを
  mats.shape == (65536, 4, 4) の uint8 配列で返す。
  """
  N = 1 << L**2  # 65536
  n_ = np.arange(N, dtype=np.uint16)               # (65536,)
  bits = ((n_[:, None] >> np.arange(16, dtype=np.uint16)) & 1).astype(np.uint8)  # (65536,16)
  mats = bits.reshape(N, 4, 4)                    # (65536,4,4)
  return mats

def add_first_row_col(a: np.ndarray) -> np.ndarray:
    if a.shape != (4,4):
        raise ValueError("入力は4x4配列にしてください。")
    # 右に1列目を追加 → (4,5)
    b = np.hstack([a, a[:, [0]]])
    # 下に1行目（※いまの b の1行目）を追加 → (5,5)
    c = np.vstack([b, b[[0], :]])
    return c

def construct_indices_to_get_checkerboard_block(n_rows):
  indices = []
  for n in range(n_rows):
    if n % 2 == 0:
      indices.append((n, 0))
      indices.append((n, 2))
    else:
      indices.append((n, 1))
      indices.append((n, 3))
  return np.array(indices)

def get_value_from_rho(spin_latice):
  if np.array_equal(spin_latice, np.array([[0, 0],[0, 0]])):
    return a_in_rho
  elif np.array_equal(spin_latice, np.array([[1, 0],[1, 0]])):
    return b_in_rho
  elif np.array_equal(spin_latice, np.array([[0, 1],[0, 1]])):
    return b_in_rho
  elif np.array_equal(spin_latice, np.array([[1, 0],[0, 1]])):
    return c_in_rho
  elif np.array_equal(spin_latice, np.array([[0, 1],[1, 0]])):
    return c_in_rho
  elif np.array_equal(spin_latice, np.array([[1, 1],[1, 1]])):
    return a_in_rho
  else:
    return 0

def product_of_partial_boltzmann_factors(spin_blocks):
  product = 1
  for block in spin_blocks:
    val = get_value_from_rho(block)
    if val == 0:
      return 0
    product *= val

  return product


mats = all_4x4_binary_matrices_array()
indices = construct_indices_to_get_checkerboard_block(mats[0].shape[0])

# m = np.array([
#   [1, 0, 1, 0],
#   [0, 1, 0, 1],
#   [1, 0, 1, 0],
#   [0, 1, 0, 1],
# ])
# mats = np.array([m])
n_non_zero = 0
sum = 0

for mat in mats:
  # 周期的境界条件のため、行の末尾に最初の行を、列の末尾に最初の列を追加
  mat = add_first_row_col(mat)

  # チェッカーボード分解したスピン行列から、2x2のブロックを全通り取り出す
  blocks = np.lib.stride_tricks.sliding_window_view(mat, (2, 2))
  # 部分ボルツマン因子を取り出すブロックだけ取る
  r, c = np.array(indices).T
  picked = blocks[r, c]

  # 対象の部分ボルツマン因子の積を計算する
  product = product_of_partial_boltzmann_factors(picked)

  if product != 0:
    n_non_zero += 1
    sum += product

# n=2, L=4の場合、90通りの電子配列で0ではなければOK
print(n_non_zero)
print('Zn=', sum)

90
Zn= 44.627744732937494


### 1次元XXXハイゼンベルグモデルをメトロポリス法でサンプリングして計算する

局所プラケット(2x2の電子配列のブロック)をフリップする方法で更新する。ランダムに白マスを選んでフリップし、その周辺の4プラケットに対してρを求めて更新確率を決める。

ある程度電子配置は更新されている模様だが、結果は$E=-0.3166987272137258$となり$Zn$を求めてから数値微分した方式と値が一致しない。どこかに誤りがあるのかも知れない。

In [39]:
import numpy as np

# パラメータ
L = 4              # 1次元スピン鎖の長さ
n = 4              # トロッター分割数（分割数x2した値をセットする）
beta = 1.0         # 逆温度
J = 1.0            # 相互作用定数
n_steps = 10000    # MCS数

# 部分ボルツマン因子ρの要素
a_in_rho = math.exp(beta*J/(2*n))
b_in_rho = math.exp(-beta*J/(2*n))*math.cosh(beta*J/n)
c_in_rho = math.exp(-beta*J/(2*n))*math.sinh(beta*J/n)

# 許されるスピン配列で初期化
def construct_initial_config(n, L):
    spins = np.ones((n, L), dtype=int)
    for i in range(L):
        spins[:,i] = 1 if i%2==0 else -1
    return spins

# 1ステップ更新
def metropolis_update(spins):
  n, L = spins.shape
  for _ in range(n*L):
    # ランダムに白マスの左上を選ぶ
    while True:
      i = np.random.randint(n)
      j = np.random.randint(L)
      if (i % 2 == 0 and j % 2 == 1) or (i % 2 == 1 and j % 2 == 0):
        break

    block = pick_plaquette(spins, i, j)
    block_flipped = -block

    # 部分ボルツマン因子の比を計算
    # print("spins(before):\n", spins)
    rho_before = product_of_neighbor_rhos(spins, i, j)
    # print("rho_before: ", rho_before)
    fliped_spins = flip_plaquette(spins, i, j)
    # print("fliped_spins:\n", fliped_spins)
    rho_after = product_of_neighbor_rhos(fliped_spins, i, j)
    # print("rho_after: ", rho_after)
    R = rho_after / rho_before
    # メトロポリス判定
    if R >= 1.0 or np.random.rand() < R:
        spins = fliped_spins
        # print("spin updated:\n", spins)
  return spins

# チェッカーボード上の左上の格子点i,jを指定すると、4格子点のプラケットを返す
def pick_plaquette(spins, i, j):
  block = np.array([
    [spins[i,j], spins[i,(j+1)%L]],
    [spins[(i+1)%n,j], spins[(i+1)%n,(j+1)%L]]
  ])
  return block

# チェッカーボード上の左上の格子点i,jを指定すると、周辺の4プラケットの部分ボルツマン因子の積を返す
def product_of_neighbor_rhos(spins, i, j):
  plaquette1 = pick_plaquette(spins, (i+1)%n, j)
  plaquette2 = pick_plaquette(spins, (i-1)%n, j)
  plaquette3 = pick_plaquette(spins, i, (j-1)%L)
  plaquette4 = pick_plaquette(spins, i, (j+1)%L)
  return get_rho(plaquette1) * get_rho(plaquette2) * get_rho(plaquette3) * get_rho(plaquette4)

def flip_plaquette(spins, i, j):
  flipped = spins.copy()
  # 4つの格子をフリップする
  flipped[i,j] *= -1
  flipped[i,(j+1)%n] *= -1
  flipped[(i+1)%L,j] *= -1
  flipped[(i+1)%L,(j+1)%n] *= -1
  return flipped

# 部分ボルツマン因子のスピン配列に対応する要素を取得する
def get_rho(plaquette, beta=None):
  a_, b_, c_ = None, None, None
  if beta is not None:
    a_ = math.exp(beta*J/(2*n))
    b_ = math.exp(-beta*J/(2*n))*math.cosh(beta*J/n)
    c_ = math.exp(-beta*J/(2*n))*math.sinh(beta*J/n)

  if np.array_equal(plaquette, np.array([[1, 1],[1, 1]])):
    return a_ or a_in_rho
  elif np.array_equal(plaquette, np.array([[1, -1],[1, -1]])):
    return b_ or b_in_rho
  elif np.array_equal(plaquette, np.array([[-1, 1],[-1, 1]])):
    return b_ or b_in_rho
  elif np.array_equal(plaquette, np.array([[1, -1],[-1, 1]])):
    return c_ or c_in_rho
  elif np.array_equal(plaquette, np.array([[-1, 1],[1, -1]])):
    return c_ or c_in_rho
  elif np.array_equal(plaquette, np.array([[-1, -1],[-1, -1]])):
    return a_ or a_in_rho
  else:
    return 0

# チェッカーボード上の黒マス全てのrhoの積を計算する
def product_of_rhos(spins):
  n, L = spins.shape
  product = 1
  for i in range(n):
    for j in range(L):
      # 黒マスのみ計算する
      if (i % 2 == 0 and j % 2 == 0) or (i % 2 == 1 and j % 2 == 1):
        plaq = pick_plaquette(spins, i, j)
        product *= get_rho(plaq)
  return product

# エネルギー計算
def energy(spins, beta):
    # 数値微分
    def drho_dbeta(plaq, beta, h=1e-5):
        return (get_rho(plaq, beta+h) - get_rho(plaq, beta-h)) / (2*h)

    sum = 0
    n, L = spins.shape
    for i in range(n):
        for j in range(L):
            # 黒マス 
            if (i % 2 == 0 and j % 2 == 0) or (i % 2 == 1 and j % 2 == 1):
                plaq = pick_plaquette(spins, i, j)
                sum += drho_dbeta(plaq, beta) / get_rho(plaq, beta)
    return -sum

# メインループ
spins = construct_initial_config(n, L)
print("spins(before):\n", spins)

energies = []
for step in range(n_steps):
    spins = metropolis_update(spins)
    energies.append(energy(spins, beta))

  # if step > n_steps//2:  # 熱平衡化後に測定
  # energies.append(energy(spins))

print("spins(after):\n", spins)
print("エネルギー期待値=", np.mean(energies))


spins(before):
 [[ 1 -1  1 -1]
 [ 1 -1  1 -1]
 [ 1 -1  1 -1]
 [ 1 -1  1 -1]]
spins(after):
 [[ 1  1 -1 -1]
 [ 1  1 -1 -1]
 [ 1  1 -1 -1]
 [ 1  1 -1 -1]]
平均エネルギー = -0.3166987272137258


### エネルギー期待値を数値微分で求める

パラメータは$n=2, L=4, \beta=1, J=1$（nは処理の都合上2nの値をパラメータに入れている）

モンテカルロ法がうまく実装できなかったので、分配関数Znの値を求めてから、$E=-\frac{\partial(lnZ_n)}{\partial \beta}$を数値微分で求めると、$E=-1.6546837361408604$となった。この条件でのエネルギー期待値は-1.6453897143 (by ChatGPT)らしいので一致している？

In [37]:
import numpy as np
import math

# パラメータ
L = 4              # 1次元スピン鎖の長さ
n = 4              # トロッター分割数（分割数x2した値をセットする）
beta = 1.0         # 逆温度
J = 1.0            # 相互作用定数

def all_possible_configurations(n, L):
    N = 1 << (n*L)   # 2^(L^2)
    n_ = np.arange(N, dtype=np.uint32)
    bits = ((n_[:, None] >> np.arange(L**2, dtype=np.uint32)) & 1).astype(np.int8)
    spins = 2*bits - 1   # 0→-1, 1→+1
    mats = spins.reshape(N, n, L)
    return mats

# チェッカーボード上の左上の格子点i,jを指定すると、4格子点のプラケットを返す
def pick_plaquette(spins, i, j):
  block = np.array([
    [spins[i,j], spins[i,(j+1)%L]],
    [spins[(i+1)%n,j], spins[(i+1)%n,(j+1)%L]]
  ])
  return block

# 全ての許容される電子配列を生成する
def all_allowed_configurations(n, L, rho_entries):
  worlds = all_possible_configurations(n, L)
  # print("worlds.shape:", worlds.shape)
  allowed_worlds = []
  for spins in worlds:
    # print("spins:\n", spins)
    if grid_check_rho_value(spins, rho_entries):
      allowed_worlds.append(spins)
  return np.array(allowed_worlds)

# 各プラケットを走査して、許される電子配列かどうかをチェックする
def grid_check_rho_value(spins, rho_entries):
  n, L = spins.shape
  for i in range(n):
    for j in range(L):
      # 黒マスのみ計算する
      if (i % 2 == 0 and j % 2 == 0) or (i % 2 == 1 and j % 2 == 1):
        plaq = pick_plaquette(spins, i, j)
        # print(plaq)
        if get_rho(plaq, rho_entries) == 0:
          return False
  return True

# チェッカーボード上の黒マス全てのrhoの積を計算する
def product_of_rhos(spins, rho_entries):
  n, L = spins.shape
  product = 1
  for i in range(n):
    for j in range(L):
      # 黒マスのみ計算する
      if (i % 2 == 0 and j % 2 == 0) or (i % 2 == 1 and j % 2 == 1):
        plaq = pick_plaquette(spins, i, j)
        product *= get_rho(plaq, rho_entries)
  return product

# 部分ボルツマン因子のスピン配列に対応する要素を取得する
def get_rho(plaquette, rho_entries):
  a_in_rho, b_in_rho, c_in_rho = rho_entries
  if np.array_equal(plaquette, np.array([[1, 1],[1, 1]])):
    return a_in_rho
  elif np.array_equal(plaquette, np.array([[1, -1],[1, -1]])):
    return b_in_rho
  elif np.array_equal(plaquette, np.array([[-1, 1],[-1, 1]])):
    return b_in_rho
  elif np.array_equal(plaquette, np.array([[1, -1],[-1, 1]])):
    return c_in_rho
  elif np.array_equal(plaquette, np.array([[-1, 1],[1, -1]])):
    return c_in_rho
  elif np.array_equal(plaquette, np.array([[-1, -1],[-1, -1]])):
    return a_in_rho
  else:
    return 0

# βを受け取ってZnを計算する
def calc_zn(n, L, beta=1.0, J=1.0):
  # 部分ボルツマン因子ρの要素
  # rho_entries = [math.exp(beta*J/(2*n)), math.exp(-beta*J/(2*n))*math.cosh(beta*J/n), math.exp(-beta*J/(2*n))*math.sinh(beta*J/n)]
  # ρの成分は2nのうちのnによって決まるので、1/2している
  n_ = n / 2
  rho_entries = [math.exp(beta*J/(2*n_)), math.exp(-beta*J/(2*n_))*math.cosh(beta*J/n_), math.exp(-beta*J/(2*n_))*math.sinh(beta*J/n_)]
  # メインループ
  worlds = all_allowed_configurations(n, L, rho_entries)
  Zn = 0
  for spins in worlds:
    product = product_of_rhos(spins, rho_entries)
    if product != 0:
      Zn += product
  print("Zn:", Zn)
  return Zn

# 数値微分
def dfdx(fn, n, L, beta, h=1e-5):
  return (fn(n, L, beta+h) - fn(n, L, beta-h)) / (2*h)

# lnZを求める
def fn(n, L, beta, h=1e-5):
  Zn = calc_zn(n, L, beta)
  return math.log(Zn)

E = -dfdx(fn, n, L, beta)
print("E:", E)

Zn: 44.628483188555606
Zn: 44.62700629248769
E: -1.6546837361408604
